# AssocDS_Recruitment_chanllenge

In [ ]:
# import packages
# Using 'pip install -r requirements.txt' to install all packages
import pandas as pd
import numpy as np
import plotly.express as px
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from itertools import product
from math import sqrt
import warnings
from prophet import Prophet

## Part A: Data preparation

In [ ]:
# Read the file 
DecemberCols = pd.read_csv('Data/DecemberCols.csv', header=None)
DecemberAdData = pd.read_csv('Data/DecemberAdData.csv')
MarketingCols = pd.read_csv('Data/MarketingCols.csv', header=None)
MarketingSales = pd.read_csv('Data/MarketingSales.csv')

In [ ]:
# Merge December columns name and Data
DecemberCols_column_name = DecemberCols.iloc[:,0].tolist()
DecemberAdData.columns =  DecemberCols_column_name
DecemberAdData.head()

In [ ]:
# Merge Market columns name and Data
Market_column_name = MarketingCols.iloc[:,0].tolist()
MarketingSales.columns =  Market_column_name
MarketingSales.head()

In [ ]:
# Check if there is NULL value in MarketingSales
missing_values_per_column = MarketingSales.isna().sum()
print(missing_values_per_column)

In [ ]:
# copy the dataframe to df 
df = MarketingSales.copy()

# Format the Date columns
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%y')
#  Extract year from the Date column
df['Year'] = df['Date'].dt.year
df.set_index('Date', inplace=True)  # Set date as the index

# Ensure the data is sorted by date
df.sort_index(inplace=True)

df.head()

## Part B: Exploratory Data Analysis (EDA)

### 1. seasonality analysis & trend

In [ ]:
# Now create the line plot, faceted by Year, with separate lines for PositiveNews
fig = px.line(df, x='Month', y='Sales', color='PositiveNews', 
              line_group='PositiveNews', # Ensure lines are grouped by  
              facet_row='Year', # Create a separate subplot for each year
              category_orders={"Month": ['January', 'February', 'March', 'April', 'May', 'June',
                                         'July', 'August', 'September', 'October', 'November', 'December']},
              title='Sales Trends by Month and PositiveNews Status Across Years')

# Update the layout for better readability
fig.update_layout(xaxis_title='Month', yaxis_title='Total Sales')
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1])) # Simplify facet titles
fig.update_xaxes(tickangle=-45)

# Show the figure
fig.show()

# Now create the line plot, faceted by Year, with separate lines for NegativeCoverage
fig = px.line(df, x='Month', y='Sales', color='NegativeCoverage', 
              line_group='NegativeCoverage', # Ensure lines are grouped by  
              facet_row='Year', # Create a separate subplot for each year
              category_orders={"Month": ['January', 'February', 'March', 'April', 'May', 'June',
                                         'July', 'August', 'September', 'October', 'November', 'December']},
              title='Sales Trends by Month and NegativeCoverage Status Across Years')

# Update the layout for better readability
fig.update_layout(xaxis_title='Month', yaxis_title='Total Sales')
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1])) # Simplify facet titles
fig.update_xaxes(tickangle=-45)

# Show the figure
fig.show()

### Positive News Trend Analysis:
- There's a noticeable pattern where **sales tend to increase towards the end of the year**, likely around the holiday season, which could suggest a **seasonal effect**.
- In years and months with Positive News, we observe a **stronger end-of-year sales increase**, which might indicate that **positive media coverage** is effectively boosting sales during this period.
- The increase in sales during the latter months, such as November and December, could be related to **promotional activities or holiday shopping behaviors**.

### Negative Coverage Trend Analysis:
- In contrast, the graph with Negative Coverage shows **less end-of-year sales spikes** compared to the Positive News graph.
- Sales appear to be more consistent throughout the year in the presence of Negative Coverage, with **fewer peaks**.
- The dampening of the end-of-year spike could suggest that **negative publicity impacts** the usual seasonal sales boosts.
- Interestingly, while Negative Coverage seems to reduce the magnitude of the seasonal peaks, it **doesn't negate the overall upward trend** towards the year's end, indicating **other factors** may also be influencing sales.

### Overall Interpretation:
- The presence of Positive News correlates with an **accentuated increase in sales towards the end of the year**, reinforcing the idea that **positive media attention can have a beneficial impact** on consumer behavior.
- Negative Coverage might **mitigate the positive impact of seasonality on sales**, but it doesn't entirely reverse the seasonal trend, suggesting that sales are influenced by a **complex set of factors** beyond media coverage.
- It's essential to consider **other variables not shown in these graphs** that could be affecting sales, such as **marketing campaigns, economic conditions, or changes in product offerings**.
- The above graph tells us that **sales tend to spike at the end of year**, this confirms that the sales vary with the ‘Date’ (time) and there is a **seasonality factor** present in our data.


### 2. Relationship Between Sales and Advertising Spend 

In [ ]:
# Plot
fig = px.scatter(df, x='AdvertisingSpend', y='Sales', title='Relationship Between Sales and Advertising Spend',
                 trendline='ols', trendline_color_override='darkblue')

# Update layout for better readability
fig.update_layout(xaxis_title='Advertising Spend', yaxis_title='Sales')

# Show the figure
fig.show()

The scatter plot above visualizes the **relationship between advertising spend and sales**. Each point on the graph represents a data record from the dataset.

### Observations:
- **Positive Correlation**: There appears to be a **positive correlation** between advertising spend and sales, as indicated by the overall upward trend in the data points. This suggests that higher advertising spend is generally associated with higher sales.
- **Data Density**: Most data points are clustered at the lower end of advertising spend, which indicates that there are more records with lower advertising budgets.
- **Trendline**: The trendline, shown in dark blue, provides a **linear approximation of the general trend** of the data. It highlights the average effect of advertising spend on sales across the dataset.

### Implications:
- **Marketing Efficiency**: The pattern observed suggests that there may be an **optimal range of advertising spend** where the return on investment in terms of sales is maximized.
- **Budget Allocation**: Companies may consider increasing their advertising budget up to a point where the sales continue to grow proportionately, beyond which the additional spend may yield **diminishing returns**.

### 3. Stationarity of Time Series

#### 3.1 Rolling Mean & Variance

In [ ]:
# Define the rolling window size
window_size = 30   # A 30-day window for monthly patterns

# Create stationarity data
# test = df[df['']]
# Calculate the rolling mean and rolling st
rolling_mean = df['Sales'].rolling(window=window_size).mean()
rolling_variance = df['Sales'].rolling(window=window_size).std()

# Plotting the original sales data, rolling mean, and rolling variance
plt.figure(figsize=(14, 7))
plt.plot(df['Sales'], label='Original Sales', color='blue')
plt.plot(rolling_mean, label='Rolling Mean', color='red')
plt.plot(rolling_variance, label='Rolling Variance', color='green')
plt.title('Sales with Rolling Mean and Standard Deviation')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.legend()
plt.show()


## Stationarity of Time Series

Analyzing the plot of sales with rolling mean and variance, we observe the following:

- The **original sales data** exhibit significant spikes, suggesting potential **seasonal patterns**. The huge drop down at early 2020 as **Covid** happen 
- The **rolling mean** appears stable, but the spikes indicate possible **seasonal effects**.
- The **rolling standard deviation** providing insight into the variability or volatility of the sales data over time.

### Conclusion on Stationarity:
We can see from the above plot and statistical test that mean and variation doesn’t change much with time, i.e they are constant. Thus, we don’t need to perform any transformation. Then we need to do statistical test to verify if it is stationarity 

#### 3.2 Dickey-Fuller test (ADF)

In [ ]:
# Perform Augmented Dickey-Fuller test
adf_result = adfuller(df['Sales'])

print('ADF Statistic:', adf_result[0])
print('p-value:', adf_result[1])
print('Critical Values:')
for key, value in adf_result[4].items():
    print(f'   {key}: {value}')

# Interpretation of the ADF results
if adf_result[0] < adf_result[4]["5%"]:
    print("The time series is stationary as the Test Statistic is lower than the 5% Critical Value.")
else:
    print("The time series is non-stationary as the Test Statistic is higher than the 5% Critical Value.")

The Augmented Dickey-Fuller (ADF) test has been conducted to determine the stationarity of the time series. The results are as follows:

- **ADF Statistic**: The calculated value is approximately `-3.66`.
- **p-value**: The test yields a p-value of approximately `0.00468`.
- **Critical Values**: The critical value for the `5%` confidence level is approximately `-2.86`.

### Stationarity Perspective:

- **ADF Statistic vs Critical Values**: The ADF statistic is **lower** than the 5% critical value, which indicates that we can **reject the null hypothesis** of a unit root (non-stationarity) in the time series data.
  
- **p-value Analysis**: With a p-value **less than 0.05**, we have strong evidence against the null hypothesis, further confirming the **stationarity** of the time series.

### Conclusion:

Based on the ADF test results, the time series is considered **stationary** at the 5% significance level, having a **constant mean and variance** over time. This indicates that the time series is free from time-dependent structures such as trends or seasonality, making it suitable for use with forecasting models that assume stationarity.

#### 3.4 decomposition plots

In [ ]:
# seasonality and trend using decomposition plots
# Perform seasonal decomposition with an additive model, assuming monthly data with a yearly cycle (period=12)
result = seasonal_decompose(df['Sales'], model='additive', period=7)

# Create data frames for trend, seasonal, and residual components from decomposition
trend = result.trend.reset_index()
seasonal = result.seasonal.reset_index()
resid = result.resid.reset_index()

# Plot the Trend Component
fig_trend = px.line(trend, x='Date', y='trend', title='Trend Component of Sales')
fig_trend.show()

# Plot the Seasonal Component
fig_seasonal = px.line(seasonal, x='Date', y='seasonal', title='Seasonal Component of Sales')
fig_seasonal.show()

# Plot the Residual Component
fig_resid = px.line(resid, x='Date', y='resid', title='Residual Component of Sales')
fig_resid.show()


The time series data has been decomposed into three main components: trend, seasonal, and residual.

- **Trend Component**: Shows a smooth progression and provides insight into the underlying long-term trend of the sales data.
- **Seasonal Component**: Illustrates the repeating patterns or cycles in the data over specific time intervals, indicating consistent seasonality.
- **Residual Component**: Represents the irregularities or 'noise' remaining in the data after the trend and seasonal components have been extracted.

### Conclusion on Stationarity:

Based on the decomposition, if the **trend component** is relatively flat and the **seasonal component** shows consistent patterns without increasing or decreasing in magnitude, it suggests that the series is stationary with respect to trend and seasonality.

The **residual component** should ideally show no pattern and have constant variance, further supporting stationarity. However, if there's significant variance or patterns in the residuals, non-stationarity may still be a concern.

Considering both the decomposed components and the ADF test results, the data can be tentatively considered stationary if all conditions are met。

## Part C: Predictive Modelling

### Model 1: SARIMAX Time Series Modeling for Sales Forecasting

In [ ]:
# Explicitly set frequency, for example daily frequency
df = df.asfreq('D')

# split into train and test dataset
train_size = int(len(df) * 0.9)
train = df.iloc[:train_size]
test = df.iloc[train_size:]

p_values = range(0, 2)
d_values = range(0, 2)
q_values = range(0, 2)
P_values = range(0, 2)
D_values = range(0, 2)
Q_values = range(0, 2)
s_values = [7]

# Grid search to find the best parameters
best_aic = float('inf')
best_params = None
for p, d, q, P, D, Q, s in product(p_values, d_values, q_values, P_values, D_values, Q_values, s_values):
    try:
        model = SARIMAX(train['Sales'], order=(p, d, q), seasonal_order=(P, D, Q, s))
        model_fit = model.fit()
        if model_fit.aic < best_aic:
            best_aic = model_fit.aic
            best_params = (p, d, q, P, D, Q, s)
    except:
        continue

# Train model using the best parameters
model = SARIMAX(train['Sales'], order=best_params[:3], seasonal_order=best_params[3:])
model_fit = model.fit()

# Print the best SARIMA model's AIC and parameters
print(f'Best SARIMA Model: AIC={best_aic:.2f}, order={best_params[:3]}, seasonal_order={best_params[3:]}')

# Plot the prediction results
plt.figure(figsize=(12, 6))
plt.plot(train['Sales'], label='Train')
plt.plot(test['Sales'], label='Test')
plt.plot(model_fit.forecast(len(test)), label='Predicted')
plt.legend()
plt.title('Time Series Prediction')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.show()

# Calculate error
mse = mean_squared_error(test['Sales'], model_fit.forecast(len(test)))
rmse = np.sqrt(mse)  # Compute root mean squared error
print(f'RMSE: {rmse:.2f}')

### Model Evaluation
- **Forecasting Accuracy**: The predicted values (green) show a projection that initially follows the test data trend but eventually starts to deviate. Notably, the predicted values at the end of the timeline do not capture the upturn observed in the test data (orange).
- **RMSE Value**: The RMSE of 15.59 measures the model's forecast error magnitude

In [ ]:
# Checking diagnostic plots
model_fit.plot_diagnostics(figsize=(10, 10))
plt.show()

- **Standardized Residuals**: The residuals plot does not show any obvious patterns or systematic structure, which is a good indication that the model is capturing the underlying process well.

- **Histogram and Estimated Density**: The histogram of standardized residuals suggests a reasonably normal distribution, closely matching the overlaid normal curve (N(0,1)). This indicates that the residuals are approximately normally distributed, meeting one of the key assumptions of many time series models.

- **Normal Q-Q Plot**: The Q-Q plot points largely follow the theoretical line, with some deviations on the tails. This indicates that while the residuals are mostly normally distributed, there may be outliers or extreme values not captured by the model.

- **Correlogram**: The ACF plot shows that almost all autocorrelations for the residuals are within the confidence interval, suggesting that there is little to no autocorrelation left in the residuals. This implies that the model is accounting well for the time series structure.

#### Conclusion
The model diagnostics suggest that the SARIMAX model performs adequately, with residuals that are close to normally distributed and no significant autocorrelation. Some minor deviations from normality are noted, which could potentially be improved with further model refinement. However, the current model appears to be a good fit for the data, capturing its main characteristics.

### Model 2: Prophet Time Series Modeling for Sales Forecasting

In [ ]:
warnings.simplefilter(action='ignore', category=FutureWarning)

# Load and prepare the data
df_prophet = MarketingSales[['Date', 'Sales']].copy()
df_prophet['Date'] = pd.to_datetime(df_prophet['Date'], format='%d/%m/%y')
df_prophet = df_prophet.rename(columns={'Date': 'ds', 'Sales': 'y'})

# Split the data into train and test sets based on the date
cutoff_date = pd.Timestamp('2020-03-01')
train = df_prophet[df_prophet['ds'] < cutoff_date]
test = df_prophet[df_prophet['ds'] >= cutoff_date]

# Create and fit the Prophet model
model = Prophet(daily_seasonality=True)
model.fit(train)

# Make a future dataframe for predictions
future_dates = model.make_future_dataframe(periods=test.shape[0])

# Predict
forecast = model.predict(future_dates)

# Plot the forecast along with the actual values
fig, ax = plt.subplots(figsize=(10, 6))  # Configure the figure size before plotting
fig = model.plot(forecast, ax=ax)
#test.plot(x='ds', y='y', ax=ax, label='Test Data', style='r-')
ax.legend()
plt.show()

# Calculate RMSE
# Extract the predicted yhat values for the test period
forecasted_test = forecast[['ds', 'yhat']].merge(test, on='ds')
rmse = sqrt(mean_squared_error(forecasted_test['y'], forecasted_test['yhat']))
print(f"RMSE: {rmse}")

# Optionally, display components of the forecast
fig2 = model.plot_components(forecast)
plt.show()


### Model Fit and Forecast Analysis
The Prophet model output showcases several key features:
- **Actual Sales Data**: Represented by the black dots, they give a clear indication of historical sales variability and seasonality.
- **Forecasted Sales**: The blue line indicates the model's predicted sales values, showing a grasp of the data's underlying patterns and trends.
- **Confidence Interval**: The shaded blue area illustrates the forecast's uncertainty, providing a visual representation of prediction confidence.

Observations from the plot:
- The seasonal peaks and troughs in the historical data are well-captured by the model's forecast.
- The model projects an upward trend in sales, suggesting a positive growth expectation. This looks much better compared with SARIMAX Model which has not able to projects an upard trend. 

### Components of the Forecast
The component plots break down the forecast into its constituent parts:
- **Trend**: A general increase over time suggests overall sales growth.
- **Weekly Seasonality**: Fluctuations in sales during the week could inform marketing strategies and operational planning.
- **Yearly/Daily Seasonality**: Insight into periods of high or low activity throughout the year or day, which can aid in long-term planning and resource allocation.

### Performance Metric
- **Root Mean Squared Error (RMSE)**: The model achieves an RMSE of approximately 19.58, which is a standard metric for evaluating the accuracy of a model's predictions. The effectiveness of the RMSE value is contingent upon the sales data scale.

### Conclusion
The Prophet model demonstrates a strong capability in capturing the sales data's patterns, with effective trend identification and seasonality projection. The RMSE indicates the model's predictions are relatively accurate, though the actual performance should be contextualized within the specific business environment.

## Part C: Forcast December 2020 Sales

We will use Prophet model to make prediction for the December 2020 Sales

In [ ]:
# Generate a future dataframe starting from December 1, 2020, for the month of December
# There are 31 days in December 2020
future = pd.date_range(start='2020-12-01', periods=31, freq='D')
future_df = pd.DataFrame(future, columns=['ds'])

# Use the model to make a forecast for December 2020
forecast = model.predict(future_df)

In [ ]:
# Extract the sales prediction for Dec
forcast_dec = forecast[['ds', 'yhat']]
# Rename 
forcast_dec = forcast_dec.rename(columns={'ds': 'Date', 'yhat': 'Sales'})

# Format the Date columns
DecemberAdData['Date'] = pd.to_datetime(DecemberAdData['Date'], format='%d/%m/%y')

In [ ]:
# December Prediction
Dec = pd.merge(DecemberAdData, forcast_dec, on='Date', how='left')
print('Sales Forecast for December 2020:')
print(Dec)

# Total expected sales for December 2020
print('\nTotal expected sales for December 2020:', Dec['Sales'].sum())

In [ ]:
# Create a new figure with two y-axes
fig, ax1 = plt.subplots(figsize=(12, 6))

# First y-axis
color = 'tab:blue'
ax1.set_xlabel('Date')
ax1.set_ylabel('Sales', color=color)
ax1.plot(Dec['Date'], Dec['Sales'], color=color)
ax1.tick_params(axis='y', labelcolor=color)

# Create the second y-axis sharing the same x-axis
ax2 = ax1.twinx()
color = 'tab:orange'
ax2.set_ylabel('Advertising Spend', color=color)
ax2.plot(Dec['Date'], Dec['AdvertisingSpend'], color=color)
ax2.tick_params(axis='y', labelcolor=color)

# Add title and grid lines
fig.tight_layout()
plt.title('Prediction Sales and actual Advertising Spend on Dec 2020')
plt.grid(True)

# Display the chart
plt.show()

## Sales Forecast Trend Analysis

The blue line on the chart represents the predicted sales over the course of December 2020. Analyzing the trend:

- The **predicted sales appear relatively stable** throughout the month, with no dramatic increases or decreases indicated by the forecasting model.
- A **slight uptrend** can be observed towards the middle of the month, which could be related to increased consumer activity during the holiday season.
- The **highest point** in the predicted sales seems to occur around mid-December, which is a common trend in retail due to the holiday shopping surge.

### Strategic Recommendations for the Company

Given the expected sales pattern:
- **Stock Inventory**: The company should ensure adequate stock levels to meet the anticipated increase in consumer demand, particularly around mid-December.
- **Marketing and Promotions**: Aligning marketing efforts with the uptick in sales could further boost revenue; consider launching targeted campaigns just before the peak period.
- **Staffing and Support**: Prepare for an influx of customer inquiries and sales activity by adjusting staffing levels and customer service resources accordingly during peak times.
- **Post-Peak Strategy**: Develop a post-peak plan to maintain customer engagement and manage excess inventory as the sales trend begins to normalize towards the end of December.